# Battery Module Structural Analysis (Python)
This notebook analyzes the SES/UTS load cases for battery segments using Python. 

## 1. Constants and Geometry

             ____________________________________________________________
            /                                                           /|
           /                     POLYCARBONATE LID                     / |
          /___________________________________________________________/  |
         |                                                           |   | <--- EneSegHeight
         |                                                           |   |      (105.6mm)
         |___________________________________________________________|   |
         |                                                           |  /
         |                                                           | / <--- EneSegmentWidth
         |___________________________________________________________|/        (81.2mm)
           <------------------- EneSegmentDepth -------------------> 
                                 (417.0mm)

         _________________________________________________________   
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ] | 
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ] | 
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ] |  (81.2mm)WIDTH
      | | [ o o ]   [ o o ]   [ o o ]   [ o o ]   [ o o ]           | 
      | |_________________________________________________________| |
      |                                                             |
        <---------------------- DEPTH (417.0mm) ------------------->

In [20]:
import math

# Conversion Constants
PSI_TO_PA = 6894.76
MM_TO_M = 1/1000
INCH_TO_M = 25.4/1000
G = 9.80665


# Module Dimensions
ene_mod_width = 20.3 * MM_TO_M
ene_seg_height = 105.6 * MM_TO_M
ene_mod_depth = 69.5 * MM_TO_M
module_weight = 0.278  # kg

# Segment Assembly
ene_segment_depth = 6 * ene_mod_depth #This represents the longest dimension of the segment
ene_segment_width = ene_mod_width * 4 #This repersents the width fromm 4 enepaq bricks side by side
#1x Enepaq bricks in height | 1 * ene_seg_height = Redundant
garolite_thickness = (1/16) * INCH_TO_M
segment_modules_weight = module_weight * 23 #total enepaq mass

# Areas Garolite - Imagine Section Views
top_cross_sec_area = ene_segment_depth * garolite_thickness * 2 #airflow "endplates" arent considerd valid area   # View Axis: Depth X, Width Y
width_cross_sec_area = ene_segment_depth * garolite_thickness * 1 #airflow "endplates" arent considerd valid area # View Axis: Depth X, Height Y
depth_cross_sec_area = (ene_segment_width * garolite_thickness * 2) + (ene_seg_height * garolite_thickness * 2)   # View Axis: Width X, Height Y

In [ ]:
#markdown and also print out safety factors
#have some way to export a table of all the inputs and safety factors in one go. 


# Bolt Geometry (M6)
dia = 6.0
pitch = 1.0
dia_pitch = dia - 0.649519 * pitch
dia_root = dia - 1.226869 * pitch
area_bolt_tensile_m2 = (math.pi / 4 * ((dia_pitch + dia_root) / 2)**2) / 1e6

# Fastener Strength
example_gr5_tensile = 827370875 # Pa
max_tensile_fastener = example_gr5_tensile * area_bolt_tensile_m2

# Safety Factors (Assumes 2 bolts share load per side)
sf_bolt_tension = (max_tensile_fastener * 2) / (segment_modules_weight * 20 * G)
sf_bolt_shear = (max_tensile_fastener * 2) / (6 * segment_modules_weight * 40 * G)

# Tear-out Strength
bolt_dist_edge = 8 / 1000
max_tearout_force = bolt_dist_edge * garolite_thickness * (38000 * PSI_TO_PA)

In [23]:
# FR4 / G10 Bond Strength
fr4_bond_strength = 2200 * PSI_TO_PA
depth_force = segment_modules_weight * 40 * G

# Bond Areas
depth_restricting_bond_area = garolite_thickness * (80 / 1000)
side_bond_area = 0.02 * garolite_thickness * 2
bond_max_force = fr4_bond_strength * (depth_restricting_bond_area + side_bond_area)

## 2. Load Cases & Bending Stress
Calculated for 20g vertical and 40g lateral loads. Crash load case as supplied by SES
$$ M_{max} = \frac{w L^2}{12}, \quad w = \frac{F}{L} \implies M_{max} = \frac{F \cdot L}{12}, \quad \sigma = \frac{M_{max} \cdot \frac{t}{2}}{I}, \quad I = \frac{W t^3}{12} \implies \sigma = \frac{F L}{2 W t^2}, \quad SF = \frac{\sigma_y}{\sigma} $$

In [ ]:
import math

ene_segment_depth = 0.4170  # 6 * 69.5mm
ene_segment_width = 0.0812  # 4 * 20.3mm
segment_modules_weight = 0.278 * 23 # Total mass of 23 modules

# 1. Loading (F = m * a)
top_force = segment_modules_weight * 20 * G

# 2. Force per unit length (w)
# This represents how many Newtons act on every meter of the segment depth
w = top_force / ene_segment_depth 

# 3. Polycarbonate Lid Properties
poly_thickness = 0.25 * INCH_TO_M
poly_strength = 88.6e6  # Pa #Average Value from 56 specimens on MatWeb
poly_modulus = 2.21e9 # Pa

# 4. Bending Stress Calculation (Fixed-Fixed Beam)
# Formula: M = (w * L^2) / 12  and with f = w*l  M = (F_total * L) / 12
m_a = (top_force * ene_segment_depth) / 12

# Moment of Inertia for the lid cross-section
i_lid = (ene_segment_width * (poly_thickness**3)) / 12

# Stress: sigma = (M * c) / I
stress_bending_lid = (m_a * (poly_thickness / 2)) / i_lid
sf_lid = poly_strength / stress_bending_lid

print(f"Force per unit length (w): {w:.2f} N/m")
print(f"Max Bending Moment (M_a): {m_a:.2f} Nm")
print(f"Lid Safety Factor (20g vertical crash load): {sf_lid:.4f}")

Force per unit length (w): 3007.37 N/m
Max Bending Moment (M_a): 43.58 Nm
Lid Safety Factor (20g vertical crash load): 1.1095


## 3. Buckling Analysis (Euler-Johnson)
This section checks if the casing fails via elastic buckling or material yielding.
$$C_c = \pi \sqrt{\frac{2E}{\sigma_y}}$$


In [21]:
# Garolite Properties
garolite_strength = 38000 * PSI_TO_PA
garolite_modulus = 2400000 * PSI_TO_PA

# Moment of Inertia for the walls
i_walls = 2 * (((ene_segment_width/2 + garolite_thickness/2)**2 * garolite_thickness * (ene_seg_height + (2 * garolite_thickness))))

# Slenderness Ratios
radius_of_gyration = math.sqrt(i_walls / depth_cross_sec_area)
slenderness_ratio = ene_segment_depth / radius_of_gyration
critical_slenderness = math.pi * math.sqrt((2 * garolite_modulus) / garolite_strength)

if slenderness_ratio < critical_slenderness:
    # Johnson Inelastic Buckling
    max_force_buckle = depth_cross_sec_area * (garolite_strength - (garolite_strength**2 * slenderness_ratio**2) / (4 * math.pi**2 * garolite_modulus))
    mode = "Johnson"
else:
    # Euler Elastic Buckling
    max_force_buckle = (math.pi**2 * garolite_modulus * i_walls) / (ene_segment_depth**2)
    mode = "Euler"

sf_buckling = max_force_buckle / width_force

print(f"Buckling Mode: {mode}")
print(f"Buckling Safety Factor: {sf_buckling:.4f}")

Buckling Mode: Johnson
Buckling Safety Factor: 9.6039
